In [16]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from get_edgar_data import ParseXML
import os
from tqdm import tqdm

### Turn filings into dataframe

In [36]:
class ParseXML:
    @staticmethod
    def calc_transactionAmounts(xmlpath):
        """Calculate the total transaction amount in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        total = 0

        if xml is None:
            return total

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        for t in nonDerivativeTransactions:
            # D for disposed or A for acquired
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            # number of shares disposed/acquired
            shares = t.find("./transactionAmounts/transactionShares/value").text
            # price
            priceRaw = t.find("./transactionAmounts/transactionPricePerShare/value")
            price = 0 if priceRaw is None else priceRaw.text
            # set prefix to -1 if derivatives were disposed. set prefix to 1 if derivates were acquired.
            prefix = -1 if action == "D" else 1
            # calculate transaction amount in $
            amount = prefix * float(shares) * float(price)
            total += amount

        return round(total, 2)

    @staticmethod
    def calc_transactionPricePerShare(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        prices = []
        shares = []
        for t in nonDerivativeTransactions:
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            shareRaw = t.find("./transactionAmounts/transactionShares/value").text
            priceRaw = t.find("./transactionAmounts/transactionPricePerShare/value")
            if not priceRaw is None:
                prices.append(float(priceRaw.text) * float(shareRaw))
                shares.append(float(shareRaw))

        if (len(prices) > 0) and (len(shares) >0) and (np.sum(shares)!=0):
            return np.sum(prices) / np.sum(shares)
        else:
            return np.nan

    @staticmethod
    def calc_transactionShares(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()

        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        shares = []
        for t in nonDerivativeTransactions:
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            shareRaw = t.find("./transactionAmounts/transactionShares/value")
            prefix = -1 if action == "D" else 1
            if not shareRaw is None:
                shares.append(prefix * float(shareRaw.text))

        if len(shares) > 0:
            return np.sum(shares)
        else:
            return np.nan

    @staticmethod
    def calc_absTransactionShares(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        shares = []
        for t in nonDerivativeTransactions:
            shareRaw = t.find("./transactionAmounts/transactionShares/value")
            if not shareRaw is None:
                shares.append(float(shareRaw.text))
        if len(shares) > 0:
            return np.sum(shares)
        else:
            return np.nan

In [37]:
def get_filed_date(filingDir):
    """
    return format: str, 20180109
    """
    import re
    with open(filingDir + "/full-submission.txt", "r") as f:
        content = f.read()
    date = re.search("FILED AS OF DATE:[\s]*[0-9]{8}", content).group()[-8:]
    return pd.Timestamp(date)

filingDir = "./sec-edgar-filings/AAP\\4\\0000921895-18-000108"
xmlpath = filingDir + "/filing-details.xml"
ParseXML.calc_transactionAmounts(xmlpath=xmlpath)

191.72

### Get EDGAR stats for AAPL

In [38]:
edgarDir = "./sec-edgar-filings/"
ticker = "BIO"
filingsDir = edgarDir + "/" + ticker + "/4/"
dates = []
transactionPricePerShare = []
transactionShares = []
absTransactionShares = []
transactionAmounts = []
filingName = []
for filingDir in os.listdir(filingsDir):
    dates.append(get_filed_date(filingsDir + "/" + filingDir))
    xmlpath = filingsDir + "/" + filingDir + "/filing-details.xml" 
    transactionPricePerShare.append(ParseXML.calc_transactionPricePerShare(xmlpath=xmlpath))
    transactionShares.append(ParseXML.calc_transactionShares(xmlpath=xmlpath))
    absTransactionShares.append(ParseXML.calc_absTransactionShares(xmlpath=xmlpath))
    transactionAmounts.append(ParseXML.calc_transactionAmounts(xmlpath=xmlpath))
    filingName.append(filingDir)

In [45]:
df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
df_stock.index[0]

'2018-01-02'

In [46]:
df_data = pd.DataFrame.from_dict({
    "date": dates,
    "transactionPricePerShare": transactionPricePerShare, 
    "transactionShares": transactionShares, 
    "absTransactionShares": absTransactionShares, 
    "transactionAmounts": transactionAmounts,
    "filingName": filingName
})
df_data = df_data.set_index('date')

In [47]:
# df_data.to_csv(f"./edgar_data/{ticker}.csv")
df_stats = df_data.groupby(level=0).sum()
df_stats['numTransactions'] = df_data.groupby("date")['transactionAmounts'].count()
# df_stats.to_csv(f"./stock_edgar_stats/{ticker}.csv")
df_stats.index = df_stats.index.strftime('%Y-%m-%d')
df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
# df_stock.merge(df_stats, left_index=True, right_index=True, how='outer').to_csv(f"./bt_stock_data/{ticker}.csv")

In [59]:
df_stock.dropna().loc['2018-01-02']

ticker               BIO
open          238.669998
high          241.979996
low           238.229996
close         241.419998
volume          169473.0
adj_factor           1.0
Name: 2018-01-02, dtype: object

In [51]:
df_stats

,transactionPricePerShare,transactionShares,absTransactionShares,transactionAmounts,numTransactions
date,,,,,
2018-01-04,0.000000,5000.0,5000.0,0.00,1
2018-01-05,227.094948,-1000.0,15500.0,-1392190.53,2
2018-01-09,824.393472,-13552.0,16164.0,-2570250.15,6
2018-01-12,0.000000,0.0,0.0,0.00,4
2018-02-01,246.398547,156037.0,240561.0,-6700676.66,8
...,...,...,...,...,...
2021-12-20,10.190000,22329.0,22329.0,227532.51,7
2021-12-21,46.330500,47533.0,107033.0,-459444.12,4
2021-12-22,12.220000,4300.0,4300.0,52546.00,1


In [54]:
df_stock.merge(df_stats, left_index=True, right_index=True, how='outer')

,ticker,open,high,low,close,volume,adj_factor,transactionPricePerShare,transactionShares,absTransactionShares,transactionAmounts,numTransactions
date,,,,,,,,,,,,
2018-01-02,BIO,NaN,242.949997,241.300003,-242.125000,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2018-01-02,BIO,238.669998,241.979996,238.229996,241.419998,169473.0,1.0,NaN,NaN,NaN,NaN,NaN
2018-01-03,BIO,NaN,253.500000,249.300003,-251.399994,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2018-01-03,BIO,242.339996,251.850006,242.339996,250.470001,233223.0,1.0,NaN,NaN,NaN,NaN,NaN
2018-01-04,BIO,NaN,251.199997,248.350006,-249.774994,0.0,1.0,0.0,5000.0,5000.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-29,BIO,747.500000,753.650024,741.380005,750.739990,62829.0,1.0,NaN,NaN,NaN,NaN,NaN
2021-12-30,BIO,NaN,761.229980,758.010010,-759.619995,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2021-12-30,BIO,755.239990,761.525024,750.280029,758.650024,57620.0,1.0,NaN,NaN,NaN,NaN,NaN


In [10]:
df_stock.describe()

,open,high,low,close,volume,adj_factor,transactionPricePerShare,transactionShares,absTransactionShares,transactionAmounts,numTransactions
count,1008.000000,1008.000000,1008.000000,1008.000000,1.008000e+03,1008.0,111.000000,1.110000e+02,1.110000e+02,1.110000e+02,111.000000
mean,159.765734,161.727519,157.597685,159.717827,1.007191e+06,1.0,406.013709,-3.856324e+05,4.056308e+05,-5.896920e+05,3.243243
std,31.579776,31.589325,31.606835,31.587788,5.205922e+05,0.0,557.824709,1.711653e+06,1.707488e+06,7.750948e+06,3.464669
min,74.970001,79.940002,71.328598,75.029999,2.700920e+05,1.0,0.000000,-1.497789e+07,2.214000e+00,-8.121640e+07,1.000000
25%,140.985001,142.817501,139.399994,140.672493,6.860390e+05,1.0,0.000000,-2.352650e+04,5.184850e+02,0.000000e+00,1.000000
50%,157.335007,159.133698,155.449997,157.315002,9.008950e+05,1.0,155.780000,2.015600e+01,3.941000e+03,0.000000e+00,1.000000
75%,170.382500,171.570007,168.162502,170.064999,1.205077e+06,1.0,485.790000,7.862300e+02,4.629600e+04,7.162193e+04,4.000000
max,241.139999,243.050003,239.500000,241.910004,5.911983e+06,1.0,1899.450000,3.758350e+05,1.497789e+07,3.105715e+06,12.000000


In [60]:
def gen_bt_stock_data(ticker, edgarDir="./sec-edgar-filings/", bt_stock_data_dir="./bt_stock_data/"):
    filingsDir = edgarDir + "/" + ticker + "/4/"
    dates = []
    transactionPricePerShare = []
    transactionShares = []
    absTransactionShares = []
    transactionAmounts = []
    filingName = []
    for filingDir in os.listdir(filingsDir):
        dates.append(get_filed_date(filingsDir + "/" + filingDir))
        xmlpath = filingsDir + "/" + filingDir + "/filing-details.xml" 
        if os.path.exists(xmlpath):
            transactionPricePerShare.append(ParseXML.calc_transactionPricePerShare(xmlpath=xmlpath))
            transactionShares.append(ParseXML.calc_transactionShares(xmlpath=xmlpath))
            absTransactionShares.append(ParseXML.calc_absTransactionShares(xmlpath=xmlpath))
            transactionAmounts.append(ParseXML.calc_transactionAmounts(xmlpath=xmlpath))
            filingName.append(filingDir)

    df_data = pd.DataFrame.from_dict({
        "date": dates,
        "transactionPricePerShare": transactionPricePerShare, 
        "transactionShares": transactionShares, 
        "absTransactionShares": absTransactionShares, 
        "transactionAmounts": transactionAmounts,
        "filingName": filingName
    })
    df_data = df_data.set_index('date')

    df_data.to_csv(f"./edgar_data/{ticker}.csv")
    df_stats = df_data.groupby(level=0).sum()
    df_stats['numTransactions'] = df_data.groupby("date")['transactionAmounts'].count()
    df_stats.to_csv(f"./stock_edgar_stats/{ticker}.csv")
    df_stats.index = df_stats.index.strftime('%Y-%m-%d')

    df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
    df_stock.dropna(inplace=True)
    df_stock.merge(df_stats, left_index=True, right_index=True, how='outer').to_csv(f"./bt_stock_data/{ticker}.csv")
    df_stock = df_stock.merge(df_stats, left_index=True, right_index=True, how='outer')
    df_stock.to_csv(f"{bt_stock_data_dir}/{ticker}.csv")



In [61]:
for file in tqdm(os.listdir("./stock_data/")):
    ticker = file[:-4]
    try:
        gen_bt_stock_data(ticker)
    except Exception as e:
        print(ticker, e)

  5%|▌         | 26/485 [01:10<19:28,  2.55s/it]

AMAT [Errno 2] No such file or directory: './sec-edgar-filings//AMAT/4//0001127602-21-030292/filing-details.xml'


  6%|▌         | 29/485 [01:13<13:34,  1.79s/it]

AME [Errno 2] No such file or directory: './sec-edgar-filings//AME/4//0001127602-19-022544/filing-details.xml'


 30%|███       | 147/485 [06:01<15:36,  2.77s/it]

...

 80%|████████  | 390/485 [17:04<03:54,  2.47s/it]

SBNY [WinError 3] The system cannot find the path specified: './sec-edgar-filings//SBNY/4/'


 98%|█████████▊| 473/485 [21:05<00:24,  2.01s/it]

WTW [WinError 3] The system cannot find the path specified: './sec-edgar-filings//WTW/4/'


100%|██████████| 485/485 [21:39<00:00,  2.68s/it]


In [78]:
df0 = pd.read_csv(f"./bt_stock_data/AAL.csv", index_col=0)
bad_tickers = []
for file in tqdm(os.listdir("./bt_stock_data/")):
       df = pd.read_csv(f"./bt_stock_data/{file}", index_col=0)
       if len(df.index) != len(df0.index):
              bad_tickers.append(file[:-4])

100%|██████████| 469/469 [00:01<00:00, 373.09it/s]


In [80]:
len(bad_tickers)

163